In [5]:
import langchain
langchain.__version__

'0.2.10'

In [6]:
from langchain.agents import initialize_agent,Tool
from langchain.llms import OpenAI 
from dotenv import load_dotenv
import os

In [7]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

### Serper API Tool

In [8]:
from langchain_community.utilities import GoogleSerperAPIWrapper

search_tool = Tool(
    name = "Serper API Search",
    func = GoogleSerperAPIWrapper().run,
    description = "Use this tool to perform web searches using Serper "
)

llm = OpenAI(temperature=0.5)

tool = [search_tool]

agent = initialize_agent(tool,llm,agent="zero-shot-react-description",verbose=True)

response = agent.run("What are the latest top 2 rating tollywood movies in 2025?")

print(response)



> Entering new AgentExecutor chain...
 I should use Serper API Search to find the answer.
Action: Serper API Search
Action Input: "top 2 rating tollywood movies 2025"
Observation: BEST TELUGU MOVIES OF 2025 · 1. Court - State Vs. A Nobody · 2. Mirai · 3. HIT: The 3rd Case · 4. Eleven · 5. Kuberaa · 6. Kingdom · 7. Thandel · 8. Sankrathiki ... Mirai. Teja Sajja, Manoj Manchu, Ritika Nayak, Shriya Saran, Jayaram, Jagapathi Babu ; Little Hearts. Mouli Tanuj, Shivani Nagaram, Rajeev Kanakala, S.S. Kanchi. Best and Popular Telugu Films of 2025 So Far · Kuberaa · #Single · HIT: The 3rd Case · Mad Square · Court - State Vs. A Nobody · Thandel · Sankrathiki ... This is a list of Telugu-language films produced in Tollywood in India that are released/scheduled to be released in the year 2025. Top Grossing Tollywood Movies - 2025 1. #TheyCallHimOG - ₹285 Cr * 2. #SankrathikiVasthunam - ₹260 Cr 3. #GameChanger - ₹186 Cr 4. Shocking: No Telugu film in IMDb's Top 10 list for 2025 so far · 1. Chhaa

### Wikepedia Search

In [9]:
from langchain_community.utilities import WikipediaAPIWrapper

wiki_search_tool = Tool(
    name = "Wikipedia Search",
    func = WikipediaAPIWrapper().run,
    description = "Use this tool to search wikipedia for information on any topic"
)

llm = OpenAI(temperature=0.5)

tool = [wiki_search_tool]

agent = initialize_agent(tool,llm,agent="zero-shot-react-description",verbose=True)

response = agent.run("Who is Elon musk, Tell me the latest news of him?")

print(response)



> Entering new AgentExecutor chain...
 Elon Musk is a well-known figure, so there should be plenty of information about him online.
Action: Wikipedia Search
Action Input: "Elon Musk"
Observation: Page: Elon Musk
Summary: Elon Reeve Musk ( EE-lon; born June 28, 1971) is a businessman and entrepreneur known for his leadership of Tesla, SpaceX, Twitter, and xAI. Musk has been the wealthiest person in the world since 2021; as of October 2025, Forbes estimates his net worth to be US$500 billion.
Born into a wealthy family in Pretoria, South Africa, Musk emigrated in 1989 to Canada; he had obtained Canadian citizenship at birth through his Canadian-born mother. He received bachelor's degrees in 1997 from the University of Pennsylvania in Philadelphia, United States, before moving to California to pursue business ventures. In 1995, Musk co-founded the software company Zip2. Following its sale in 1999, he co-founded X.com, an online payment company that later merged to form PayPal, which was

### SQLite database

In [20]:
import sqlite3

connection = sqlite3.connect("example_new.db")

cursor = connection.cursor()

cursor.execute("""
create table if not exists inventory(
        id INTEGER PRIMARY KEY,
        product_name TEXT NOT NULL,
        stock_quantity INTEGER NOT NULL
        )
""")

cursor.executemany(
    """
   INSERT INTO inventory (product_name,stock_quantity) values (?,?)
""",
[
    ("Product A",150),
    ("Product B",200),
    ("Product C",300),
    ("Product D",100),
    ("Product E",50)
]
)
connection.commit()
connection.close()

In [18]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

In [22]:
db = SQLDatabase.from_uri("sqlite:///example_new.db")

llm = OpenAI(temperature=0.5)

toolkit = SQLDatabaseToolkit(db=db,llm=llm)

tools = toolkit.get_tools()

agent = initialize_agent(tools,llm,agent="zero-shot-react-description",verbose=True)

query = "How many units of Product B are in stock?"

response = agent.run(query)
print(response)



> Entering new AgentExecutor chain...
 I need to query the database to get the number of units of Product B in stock.
Action: sql_db_query
Action Input: SELECT units FROM products WHERE name = 'Product B'
Observation: Error: (sqlite3.OperationalError) no such table: products
[SQL: SELECT units FROM products WHERE name = 'Product B']
(Background on this error at: https://sqlalche.me/e/20/e3q8)
Thought: I need to use sql_db_list_tables to get a list of tables in the database.
Action: sql_db_list_tables
Action Input: ""
Observation: inventory
Thought: I need to use sql_db_schema to get the schema and sample rows for the inventory table.
Action: sql_db_schema
Action Input: inventory
Observation: 
CREATE TABLE inventory (
	id INTEGER, 
	product_name TEXT NOT NULL, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from inventory table:
id	product_name	stock_quantity
1	Product A	150
2	Product B	200
3	Product C	300
*/
Thought: I now know the table name and column names to us

### ArxivQueryRun

In [23]:
from langchain.agents import initialize_agent,Tool
from langchain.llms import OpenAI 
from langchain_community.tools.arxiv.tool import ArxivQueryRun

In [25]:
arxiv_tool = Tool(
    name = "Arxiv Search",
    func = ArxivQueryRun().run,
    description = "Use this tool to search academic papers on arxiv.Provide topics,authors or keywords"
)

llm = OpenAI(temperature=0.5)

tools = [arxiv_tool]

agent = initialize_agent(
    tools = tools,
    llm = llm,
    agent = "zero-shot-react-description",
    verbose=True
)

query = "Search for research papers on Diffusion models in 2025"

response = agent.run(query)
print("Response from agent: ")

print(response)




> Entering new AgentExecutor chain...
 I should use the Arxiv Search tool
Action: Arxiv Search
Action Input: keywords: 'Diffusion models', metadata: {'year': 2025}
Observation: Published: 2020-06-06
Title: Non Proportional Odds Models are Widely Dispensable -- Sparser Modeling based on Parametric and Additive Location-Shift Approaches
Authors: Gerhard Tutz, Moritz Berger
Summary: The potential of location-shift models to find adequate models between the
proportional odds model and the non-proportional odds model is investigated. It
is demonstrated that these models are very useful in ordinal modeling. While
proportional odds models are often too simple, non proportional odds models are
typically unnecessary complicated and seem widely dispensable. The class of
location-shift models is also extended to allow for smooth effects. The
additive location-shift model contains two functions for each explanatory
variable, one for the location and one for dispersion. It is much sparser than
ha

### Local File agents

In [26]:
from langchain_community.tools.file_management import ReadFileTool
from langchain.agents import initialize_agent,Tool
from langchain.llms import OpenAI 



In [28]:
read_file_tool = Tool(
    name = "Read File",
    func = ReadFileTool().run,
    description = "Use this tool to read the content of a text file in the current working directory.Provide the file name as input"
)

llm = OpenAI(temperature=0.5)

tools = [read_file_tool]

agent = initialize_agent(
    tools = tools,
    llm = llm,
    agent = "zero-shot-react-description",
    verbose=True
)

file_name = "example.txt"

response = agent.run(f"Read the content of the file named {file_name}.")
print(response)



> Entering new AgentExecutor chain...
 I should use the Read File tool to read the content of the file.
Action: Read File
Action Input: example.txt
Observation: Narendra Damodardas Modi[a] (born 17 September 1950) is an Indian politician who has served as the prime minister of India since 2014. Modi was the chief minister of Gujarat from 2001 to 2014 and is the member of parliament (MP) for Varanasi. He is a member of the Bharatiya Janata Party (BJP) and of the Rashtriya Swayamsevak Sangh (RSS), a right-wing Hindutva paramilitary volunteer organisation. He is the longest-serving prime minister outside the Indian National Congress.

Modi was born and raised in Vadnagar, Bombay State (present-day Gujarat), where he completed his secondary education. He was introduced to the RSS at the age of eight, becoming a full-time worker for the organisation in Gujarat in 1971. The RSS assigned him to the BJP in 1985, and he rose through the party hierarchy, becoming general secretary in 1998.[b] 